In [1]:
#all imports
import carla #the sim library itself
import random #to pick random spawn point
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import base64 # to work with API
import requests # to work with API
import json # to work with API
import pygame # to control vehicle
import time # to get time
import os # to work with saving images in folder
import threading # to work with threads

pygame 2.5.2 (SDL 2.28.3, Python 3.7.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# connect to the sim 
client = carla.Client('localhost', 2000)

In [3]:
#define environment/world and get possible places to spawn a car
world = client.get_world()
spawn_points = world.get_map().get_spawn_points()
#look for a blueprint of Mini car
vehicle_bp = world.get_blueprint_library().filter('*mini*')
#spawn a car in a random location

start_point = spawn_points[0]
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)

# move simulator view to the car
spectator = world.get_spectator()
start_point.location.z = start_point.location.z+1 #start_point was used to spawn the car but we move 1m up to avoid being on the floor
spectator.set_transform(start_point)


In [5]:
#setting Front Camera

#camera mount offset on the car
CAMERA_POS_Z = 1.6
CAMERA_POS_X = 0.9

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640') 
camera_bp.set_attribute('image_size_y', '360')

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}
camera.listen(lambda image: camera_callback(image,camera_data))

In [6]:
#setting Third Person View Camera

#camera mount offset on the car
CAMERA_POS_Z1 = 2.5 
CAMERA_POS_X1 = -6

camera_bp1 = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp1.set_attribute('image_size_x', '640')
camera_bp1.set_attribute('image_size_y', '720')

camera_init_trans1 = carla.Transform(carla.Location(z=CAMERA_POS_Z1,x=CAMERA_POS_X1))
camera1 = world.spawn_actor(camera_bp1,camera_init_trans1,attach_to=vehicle)

def camera_callback1(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w1 = camera_bp1.get_attribute('image_size_x').as_int()
image_h1 = camera_bp1.get_attribute('image_size_y').as_int()

camera_data1 = {'image': np.zeros((image_h1,image_w1,4))}
camera1.listen(lambda image: camera_callback1(image,camera_data1))

In [7]:
# Pops up windows for view camera data
def cam_stuff():
    while True:
        cv2.imshow('Front Camera View', camera_data['image'])
        cv2.imshow('Third Person View', camera_data1['image'])
        if cv2.waitKey(1) == ord('q'):
            break

    cv2.destroyAllWindows()


In [8]:
#Gemini and carla Communication and Control Script

def gemini_stuff():
    pygame.init()
    screen = pygame.display.set_mode((10, 10))
    # Create the folder if it doesn't exist
    folder_name = "step_01"
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    try:
        counter=0
        while True:
            # Assuming camera_data is updated within the loop
            img = camera_data['image']
            
            # Convert the image to numpy array
            image = np.uint8(img)

            # Save the image to a file
            file_path = "output.png"
            cv2.imwrite(file_path, image)

            # Read the saved image file and encode it as base64
            with open(file_path, "rb") as image_file:
                encoded_image = base64.b64encode(image_file.read()).decode('utf-8')

            # Prepare the payload with the base64-encoded image
            payload = {"image": encoded_image}

            
            # Send the POST request with the payload
            start_time = time.time()
            response = requests.post('https://orbital-outpost-419022.uc.r.appspot.com/api/gem', json=payload)
            geminiResponse = response.text.strip()
            print("My program took", time.time() - start_time, "to run")
            print(response.text)
            
            # Save the image again with the specified naming in the folder...this is optional to view results later
            counter += 1
            new_file_path = os.path.join(folder_name, f"{response.text}_{counter}.png")
            cv2.imwrite(new_file_path, image)

            
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    quit()

            keys = pygame.key.get_pressed()

            # Control the vehicle
            control = carla.VehicleControl()
            if geminiResponse == "STOP":
                print("yes!")
                control.brake = 1.0
            else:            
                control.throttle = 0.25

            # Apply control to the vehicle
            vehicle.apply_control(control)
            
            # Clear the screen
            screen.fill((0, 0, 0))

            pygame.display.flip()

    finally:
        # Destroy the vehicle
        vehicle.destroy()

        # Close the OpenCV windows
        cv2.destroyAllWindows()


In [16]:
# To enhance efficiency and convenience, 
# a traffic light script has been integrated into the existing setup, 
# ensuring that the first light with ID 21 remains green 
# and the second light with ID 23 stays red.
# In case there are any changes in the IDs, 
# utilize the provided script to locate the correct traffic signal post.


In [9]:
# Find the traffic light you want to locate (you need to know its ID)
traffic_light_id = 23  # Replace 9 with the ID of the specific traffic light
traffic_light = world.get_actor(traffic_light_id)

# Check if the actor is a traffic light
if traffic_light.type_id.startswith('traffic.traffic_light'):
    # Get the location of the traffic light
    traffic_light_location = traffic_light.get_transform().location
    print("Traffic light location:", traffic_light_location)

    # Set the spectator view to the location of the traffic light
    spectator = world.get_spectator()
    spectator.set_transform(carla.Transform(traffic_light_location + carla.Location(z=50), carla.Rotation(pitch=-90)))

    # Set the traffic light state to yellow
    traffic_light.set_state(carla.TrafficLightState.Yellow)
    print("Signal set to yellow.")
else:
    print("Actor is not a traffic light.")


Traffic light location: Location(x=115.449829, y=35.044945, z=0.222726)
Signal set to yellow.


In [10]:
# Get the traffic light you want to control (you need to know its ID)
traffic_light_id = 23  # Replace 10 with the ID of the specific traffic light
traffic_light = world.get_actor(traffic_light_id)

# Check if the actor is a traffic light
if traffic_light.type_id.startswith('traffic.traffic_light'):
    # Set the traffic light state to red
    traffic_light.set_state(carla.TrafficLightState.Red)
    print("Signal set to red.")

    # Disable automatic behavior
    traffic_light.set_green_time(3600.0)  # Set green time to 0
    traffic_light.set_yellow_time(0.0)  # Set yellow time to 0
    traffic_light.set_red_time(0.0)  # Set red time to a large value to make it stay red indefinitely
    print("Automatic behavior disabled. Traffic light will stay red.")
else:
    print("Actor is not a traffic light.")


Signal set to red.
Automatic behavior disabled. Traffic light will stay red.


In [11]:
# Get the traffic light you want to control (you need to know its ID)
traffic_light_id = 21  # Replace 10 with the ID of the specific traffic light
traffic_light = world.get_actor(traffic_light_id)

# Check if the actor is a traffic light
if traffic_light.type_id.startswith('traffic.traffic_light'):
    # Set the traffic light state to red
    traffic_light.set_state(carla.TrafficLightState.Green)
    print("Signal set to red.")

    # Disable automatic behavior
    traffic_light.set_green_time(0.0)  # Set green time to 0
    traffic_light.set_yellow_time(0.0)  # Set yellow time to 0
    traffic_light.set_red_time(3600.0)  # Set red time to a large value to make it stay red indefinitely
    print("Automatic behavior disabled. Traffic light will stay green.")
else:
    print("Actor is not a traffic light.")


Signal set to red.
Automatic behavior disabled. Traffic light will stay green.


In [17]:
thread1 = threading.Thread(target=cam_stuff)
thread2 = threading.Thread(target=gemini_stuff)

# Start threads
thread1.start()
thread2.start()

# Join threads (wait for them to finish)
thread1.join()
thread2.join()

My program took 5.030256271362305 to run
 GO
My program took 3.689763069152832 to run
 GO
My program took 3.314310312271118 to run
 GO
My program took 3.1978585720062256 to run
 GO
My program took 3.291590452194214 to run
 GO
My program took 3.2808761596679688 to run
 GO
My program took 3.563220977783203 to run
 STOP
yes!
My program took 3.4888243675231934 to run
 STOP
yes!
My program took 3.3909990787506104 to run
 STOP
yes!
My program took 3.421562910079956 to run
 STOP
yes!
My program took 3.490198850631714 to run
 STOP
yes!
My program took 3.1268351078033447 to run
 STOP
yes!
My program took 3.3799338340759277 to run
 STOP
yes!
My program took 3.424872636795044 to run
 STOP
yes!
My program took 3.741685390472412 to run
 GO
My program took 3.476411819458008 to run
 STOP
yes!
My program took 3.668287992477417 to run
 GO
My program took 3.4585652351379395 to run
 GO
My program took 3.488862991333008 to run
 GO
My program took 3.8090028762817383 to run
 GO
My program took 3.44444060325

Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\91986\anaconda3\envs\carla-sim\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\91986\anaconda3\envs\carla-sim\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\91986\AppData\Local\Temp\ipykernel_15796\2286006215.py", line 49, in gemini_stuff
    keys = pygame.key.get_pressed()
pygame.error: video system not initialized

